In [1]:
from tensorflow import keras

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

mymodel=keras.Sequential()
mymodel.add(keras.layers.Flatten(input_shape=[32,32,3]))

for _ in range(20):
    mymodel.add(keras.layers.Dense(100,activation='selu',kernel_initializer='lecun_normal'))
    

mymodel.add(keras.layers.AlphaDropout(rate=0.1))
mymodel.add(keras.layers.Dense(10,activation='softmax'))



In [2]:
(trainxfull,trainyfull),(testx,testy)=keras.datasets.cifar10.load_data()

In [3]:
trainx=trainxfull[5000:]
trainy=trainyfull[5000:]
vaildx=trainxfull[:5000]
vaildy=trainyfull[:5000]

In [4]:
optimizer=keras.optimizers.Nadam(lr=5e-5)
earlystop=keras.callbacks.EarlyStopping(patience=20)


In [5]:
mymodel.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [7]:
import os
import time
modelcheckpoint=keras.callbacks.ModelCheckpoint('mymodel.h5',save_best_only=True)

rootlogdir=os.path.join(os.curdir,"mylog")

def getrunlogdir():
    import time
    runid=time.strftime("run %Y_%m_%d-%H_%M_%S")
    return os.path.join(rootlogdir,runid)
runlogdir=getrunlogdir()




tenboard=keras.callbacks.TensorBoard(runlogdir)
%load_ext tensorboard
%tensorboard --logdir=.mylog --port=6006


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8008), started 19:52:08 ago. (Use '!kill 8008' to kill it.)

In [8]:
batch_size=128
rates,losses=find_learning_rate()



NameError: name 'find_learning_rate' is not defined

In [55]:
history=mymodel.fit(trainx,trainy,epochs=100,callbacks=[earlystop,modelcheckpoint,tenboard],validation_data=(vaildx,vaildy))

model=keras.models.load_model('mymodel.h5')
model.evaluate(vaildx,vaildy)

157/157 [==============================] - 1s 2ms/step - loss: 1.6255 - accuracy: 0.4130


[1.625525951385498, 0.4129999876022339]